In [ ]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
from google.colab import files

def get_wikidata_id(name):
# function to retrieve Wikidata ID for a given name
    api_url = f'https://en.wikipedia.org/w/api.php?action=query&titles={name}&prop=pageprops&format=json'
    response = requests.get(api_url).json()
    try:
        page_id = next(iter(response['query']['pages'].values()))['pageprops']['wikibase_item']
        return page_id
    except:
        return None

def get_wikidata_info(wikidata_id,data_names):
# function to retrieve Wikidata information for a given Wikidata ID and add it to a list of names with information
  url = f"https://www.wikidata.org/wiki/{wikidata_id}"
  html = requests.get(url).content.decode('utf-8')
  soup = BeautifulSoup(html, 'html.parser')
  l = soup.findAll('a')
# find the name of the entry
  name=soup.findAll('span',{'class':'wikibase-title-label'})
# find the Wikidata ID of the entry
  wikidataID=soup.findAll('span',{'class':'wikibase-title-id'})
  nameN=name[0].text
# find the English description of the entry
  discrip=soup.findAll("span",{'class':'wikibase-descriptionview-text'})
  description=discrip[0].text
# Find the div element that contains the sitelinks for the Wikidata item
  entries_div=soup.findAll("div",{"class":"wikibase-sitelinkgroupview"})
  entries=entries_div[0].findAll("li",{"class":"wikibase-sitelinkview"})

  for i in entries:
# Loop through each sitelink list item and extract its language shortcut and linked page name
    lang=i.findAll("span",{'class':'wikibase-sitelinkview-siteid'})
    wiki=lang[0].text.index('wiki')
    lang_shortcut=lang[0].text[:wiki]
    
    
    nameinlang=i.findAll('a')
    
    result=(nameN,wikidataID[0].text,description,lang_shortcut,lang[0].get('title'),nameinlang[0].text)
    data_names.append(result)


def init_full_name(NamePages):
# Create the URL for the given category
  url = f'https://en.wikipedia.org/wiki/Category:{NamePages}'
  names_with_info=[]

  while True:
# Loop through all pages in the category
      response = requests.get(url)
      html_content = response.text
      soup = BeautifulSoup(html_content, 'html.parser')
# Get all name links on the page
      name_links = soup.select('.mw-category.mw-category-columns a')
# Filter out links that are not relevant
      if NamePages=="Given_names":
        filtered_links = [link for link in name_links if not any(substring in link['href'] for substring in ['/wiki/Given_name', '/wiki/Template:R_from_given_name', '/wiki/List_of_most_popular_given_names', '/wiki/Onomancy']) and not link.find_parent(attrs={'id': 'mw-subcategories'})]
      if NamePages=="Surnames":
        filtered_links = [link for link in name_links if not any(substring in link['href'] for substring in ['/wiki/Name_blending', '/wiki/One-name_study', '/wiki/Template:R_from_surname', '/wiki/Template:Surname']) and not link.find_parent(attrs={'id': 'mw-subcategories'})]
      for link in filtered_links:
        full_name = link.text.strip()
        wikidata_info = get_wikidata_id(full_name)
        if wikidata_info:
# Get additional information about the name from Wikidata
          get_wikidata_info(wikidata_info, names_with_info)

# Check if there is a link to the next page
      next_link = soup.find('a', text='next page')
      if not next_link:
          break

      # Update the URL to the next page
      url = 'https://en.wikipedia.org' + next_link['href']

# Create a Pandas DataFrame from the list of names with information
  results_df = pd.DataFrame(names_with_info, columns=['Label', 'WikiData ID', 'English Description', 'Language','Wiki ShortLang','Entry'])
  results_df.to_csv('surname.csv', index=False, encoding='utf-8-sig')
  files.download('surname.csv')
  # results_df.to_csv('name.csv', index=False, encoding='utf-8-sig')
  # files.download('name.csv')

# NamePages="Given_names"
NamePages1="Surnames"
# init_full_name(NamePages)
init_full_name(NamePages1)